# Geospatial Transforms 

## Author(s)

Tyler Sutterley, Hannah Besso, Scott Henderson, David Shean

## Existing Notebooks

[Geospatial Transforms](https://icesat-2-2022.hackweek.io/tutorials/geospatial/geospatial-intro.html)

## Learning Outcomes

* Review fundamental concepts of geospatial coordinate reference systems (CRS)

* Learn how to access CRS metadata

* Learn basic coordinate transformations relevant to ICESat-2

### Notes

Wondering if some of this could be added to Pythia Foundations notebooks. 
The "relevance to ICESat-2" content seems particularly valuable.
